In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, collect_list, mean, stddev

# Configurar SparkSession
spark = SparkSession.builder.appName("Desviacion").getOrCreate()

# Datos de prueba
conjunto_datos = [
    ("presente", "Big Data", "Alumno1"),
    ("tarde", "Administracion de Sistemas", "Alumno2"),
    ("ausente", "Big Data", "Alumno2"),
    ("presente", "Empresa y Legislacion", "Alumno4"),
    ("ausente", "Ingenieria del Software", "Alumno5"),
    ("tarde", "Ingenieria del Software", "Alumno33"),
]
columnas = ["estado_asistencia", "asignatura", "alumno"]
df = spark.createDataFrame(conjunto_datos, columnas)

## PRUEBA 1

In [32]:
# Test: Número de alumnos por estado de asistencia
def test_totales_por_estado():
    # NÚMERO ALUMNOS POR ESTADO DE ASISTENCIA CÓDIGO DE IVÁN
    totales_por_estado = df.groupBy("estado_asistencia").agg(count("alumno").alias("total"))
    resultado = totales_por_estado.collect()

    # CÓMO LO ESTAMOS HACIENDO CON UN CONJUNTO DE DATOS DIFERENTE DE PRUEBA MARCAMOS LOS VALORES ESPERADOS QUE HEMOS INTRODUCIDO NOSOTROS MANUALMENTE
    esperado = {"presente": 2, "tarde": 2, "ausente": 2}

    # VERIFICAMOS QUE COINCIDA. ASSERT INDICARA ERROR SI ALGO NO COINCIDE
    for row in resultado:
        assert row["total"] == esperado[row["estado_asistencia"]], f"Error en estado: {row['estado_asistencia']}"

In [33]:
# EJECUTAMOS PRUEBA
test_totales_por_estado()
print("test_totales_por_estado pasó correctamente.")

test_totales_por_estado pasó correctamente.


## Prueba 2

In [36]:
def test_lista_por_asistencia():
    # LISTA DE ALUMNOS CLASIFICADOS POR ASISTENCIA CÓDIGO DE IVÁN
    lista_por_asistencia = df.groupBy("estado_asistencia").agg(collect_list("alumno").alias("lista_alumnos"))
    resultado = lista_por_asistencia.collect()

    # VOLVEMOS A MARCAR LO ESPERADO
    esperado = {
        "presente": ["Alumno1", "Alumno4"],
        "tarde": ["Alumno2", "Alumno33"],
        "ausente": ["Alumno2", "Alumno5"]
    }

    # COMPARAMOS
    for row in resultado:
        estado = row["estado_asistencia"]
        lista_actual = row["lista_alumnos"]
        lista_esperada = esperado[estado]
        
        assert set(lista_actual) == set(lista_esperada), f"Error en estado: {estado}"

In [37]:
test_lista_por_asistencia()
print("test_lista_por_asistencia pasó correctamente.")

test_lista_por_asistencia pasó correctamente.


## Prueba 3

In [24]:
# Hacemos una comprobación manual de que los datos son correctos. 
def test_media_desviacion():
    # TOTALES POR ASIGNATURA Y ESTADO DE ASISTENCIA
    totales = {}
    for row in df.collect():
        asignatura = row["asignatura"]
        totales[asignatura] = totales.get(asignatura, []) + [row["estado_asistencia"]]

    # TOTALES POR ASIGNATURA PARA CADA ESTADO
    alumnos_asignatura_estado = []
    for asignatura, estados in totales.items():
        for estado in set(estados):
            total = estados.count(estado)
            alumnos_asignatura_estado.append({"asignatura": asignatura, "estado_asistencia": estado, "total": total})

    # Calculamos la desviación
        # (Xi - media de x) al cuadrado. Por cada valor de totales se resta la media y luego elevamos al cuadrado [90 - 83.67] [85 - 83.67] [76 - 83.67]
        # Lo elevamos al cuadrado
        # Sumamos los cuadrados
        # Dividimos entre la cantidad de valores [len(totales)]
        # Hacemos la raiz cuadrada
    resultados = {}
    for asignatura in totales.keys():
        totales_por_estado = [entry["total"] for entry in alumnos_asignatura_estado if entry["asignatura"] == asignatura]
        media = sum(totales_por_estado) / len(totales_por_estado)
        varianza = sum((x - media) ** 2 for x in totales_por_estado) / len(totales_por_estado)
        desviacion = varianza ** 0.5 if len(totales_por_estado) > 1 else None
        resultados[asignatura] = {"media": media, "desviación": desviacion}

    # Definir los valores esperados
    esperado_media = {
        "Big Data": 1.0,
        "Administracion de Sistemas": 1.0,
        "Empresa y Legislacion": 1.0,
        "Ingenieria del Software": 1.0
    }
    esperado_desviacion = {
        "Big Data": 0.0,
        "Administracion de Sistemas": None,
        "Empresa y Legislacion": None,
        "Ingenieria del Software": 0.0
    }

    # VALIDAMOS
    for asignatura, valores in resultados.items():
        assert round(valores["media"], 1) == esperado_media[asignatura], f"Error en media de {asignatura}"
        if esperado_desviacion[asignatura] is None:
            assert valores["desviación"] is None, f"Error en desviación de {asignatura}"
        else:
            assert round(valores["desviación"], 1) == esperado_desviacion[asignatura], f"Error en desviación de {asignatura}"

In [25]:
test_media_desviacion()
print("test_media_desviacion pasó correctamente.")

test_media_desviacion pasó correctamente.
